In [1]:
!pip install -U transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.4/503.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


## Local Inference on GPU
Model page: https://huggingface.co/google/flan-t5-large

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/flan-t5-large)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
!nvidia-smi   # check if GPU is available

!pip install -q transformers datasets peft accelerate bitsandbytes


Tue Sep 16 02:46:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import torch
from peft import LoraConfig, get_peft_model


In [5]:
!pip install -U bitsandbytes

In [6]:
model_name = "google/flan-t5-large"   # use flan-t5-base if GPU is small

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    load_in_8bit=True,   # saves memory
    device_map="auto"
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [7]:
import pandas as pd
import random
from itertools import product

# Define subjects and their topics
subjects_topics = {
    "Machine Learning": [
        "gradient descent", "neural networks", "support vector machines", "decision trees",
        "random forests", "k-means clustering", "linear regression", "logistic regression",
        "reinforcement learning", "deep learning", "convolutional neural networks", "LSTM networks",
        "feature selection", "cross-validation", "overfitting", "regularization", "ensemble methods",
        "naive bayes", "principal component analysis", "dimensionality reduction"
    ],
    "Deep Learning": [
        "backpropagation", "activation functions", "CNN architectures", "RNN structures",
        "transformer models", "attention mechanisms", "BERT models", "GPT architecture",
        "autoencoders", "GANs", "transfer learning", "batch normalization", "dropout techniques",
        "weight initialization", "optimization algorithms", "loss functions", "computer vision",
        "natural language processing", "sequence modeling", "image classification"
    ],
    "Biology": [
        "photosynthesis", "cellular respiration", "DNA replication", "protein synthesis",
        "mitosis and meiosis", "genetics", "evolution", "ecology", "enzyme kinetics",
        "cell membrane structure", "gene expression", "molecular biology", "biotechnology",
        "immunology", "microbiology", "plant biology", "animal physiology", "biochemistry",
        "developmental biology", "neurobiology"
    ],
    "Chemistry": [
        "organic reactions", "acid-base chemistry", "electrochemistry", "thermodynamics",
        "kinetics", "equilibrium", "molecular orbital theory", "spectroscopy", "crystallography",
        "polymer chemistry", "analytical chemistry", "inorganic compounds", "coordination complexes",
        "stereochemistry", "reaction mechanisms", "catalysis", "environmental chemistry",
        "pharmaceutical chemistry", "materials science", "quantum chemistry"
    ],
    "Physics": [
        "electromagnetism", "quantum mechanics", "thermodynamics", "optics", "mechanics",
        "wave physics", "nuclear physics", "particle physics", "relativity", "condensed matter",
        "statistical mechanics", "fluid dynamics", "solid state physics", "atomic physics",
        "astrophysics", "plasma physics", "acoustics", "semiconductor physics",
        "magnetic materials", "superconductivity"
    ],
    "Mathematics": [
        "calculus", "linear algebra", "differential equations", "probability", "statistics",
        "number theory", "abstract algebra", "real analysis", "complex analysis", "topology",
        "discrete mathematics", "graph theory", "optimization", "numerical methods",
        "mathematical modeling", "combinatorics", "set theory", "logic", "geometry", "trigonometry"
    ],
    "Computer Science": [
        "algorithms", "data structures", "database systems", "software engineering", "computer networks",
        "operating systems", "computer graphics", "artificial intelligence", "cybersecurity",
        "distributed systems", "compiler design", "human-computer interaction", "web development",
        "mobile computing", "cloud computing", "parallel processing", "computer architecture",
        "formal methods", "programming languages", "information retrieval"
    ],
    "Economics": [
        "macroeconomics", "microeconomics", "econometrics", "game theory", "behavioral economics",
        "international trade", "monetary policy", "fiscal policy", "market structures", "inflation",
        "unemployment", "economic growth", "development economics", "environmental economics",
        "labor economics", "public economics", "financial economics", "industrial organization",
        "economic history", "economic policy"
    ],
    "Psychology": [
        "cognitive psychology", "behavioral psychology", "developmental psychology", "social psychology",
        "abnormal psychology", "neuropsychology", "research methods", "statistics in psychology",
        "personality psychology", "learning theories", "memory and cognition", "perception",
        "motivation and emotion", "psychological testing", "therapy techniques", "group dynamics",
        "psychological disorders", "brain and behavior", "educational psychology", "forensic psychology"
    ],
    "Statistics": [
        "descriptive statistics", "inferential statistics", "hypothesis testing", "regression analysis",
        "ANOVA", "experimental design", "sampling methods", "probability distributions",
        "Bayesian statistics", "time series analysis", "multivariate analysis", "statistical modeling",
        "data mining", "machine learning statistics", "survival analysis", "quality control",
        "survey methodology", "categorical data analysis", "non-parametric methods", "bootstrapping"
    ]
}

# Content templates for generating varied notes
intro_phrases = [
    "Today's lecture on {} provided a comprehensive examination of",
    "The session covering {} included detailed analysis of",
    "This class focused extensively on {} with particular emphasis on",
    "The comprehensive lecture on {} explored in depth",
    "Today's detailed discussion of {} covered multiple aspects including",
    "The intensive session on {} provided thorough coverage of",
    "This advanced lecture on {} examined various components of",
    "The in-depth class on {} analyzed fundamental concepts of",
    "Today's extensive coverage of {} included detailed explanations of",
    "The comprehensive session on {} provided extensive analysis of"
]

content_expansions = [
    "mathematical foundations and theoretical principles",
    "practical applications and real-world implementations",
    "algorithmic approaches and computational methods",
    "experimental procedures and laboratory techniques",
    "historical development and current research trends",
    "comparative analysis with related concepts",
    "problem-solving strategies and methodological approaches",
    "empirical evidence and statistical validation",
    "case studies and practical examples",
    "interdisciplinary connections and applications",
    "technical specifications and implementation details",
    "performance metrics and evaluation criteria",
    "optimization techniques and efficiency considerations",
    "error analysis and troubleshooting methods",
    "advanced concepts and cutting-edge research"
]

detailed_descriptions = [
    "The professor demonstrated step-by-step procedures with multiple worked examples, showing how theoretical concepts translate into practical applications.",
    "Extensive mathematical derivations were presented, including proofs and formal explanations of underlying principles.",
    "Interactive demonstrations and simulations helped illustrate complex concepts with visual representations and hands-on exercises.",
    "Comparative analysis with alternative approaches highlighted advantages, disadvantages, and appropriate use cases for different methods.",
    "Historical context was provided, tracing the development of concepts from early discoveries to modern applications.",
    "Laboratory techniques and experimental methodologies were explained with emphasis on proper procedures and safety considerations.",
    "Statistical analysis and data interpretation methods were covered, including hypothesis testing and significance evaluation.",
    "Computational implementation was discussed with programming examples and algorithm optimization strategies.",
    "Real-world case studies demonstrated practical applications across various industries and research domains.",
    "Current research trends and future directions were explored, including emerging technologies and methodological innovations.",
    "Problem-solving frameworks were established with systematic approaches to tackling complex challenges.",
    "Quality control measures and validation techniques were emphasized to ensure reliable and reproducible results.",
    "Interdisciplinary connections were highlighted, showing how concepts apply across different fields and domains.",
    "Performance evaluation metrics were introduced with benchmarking strategies and comparative assessment methods.",
    "Advanced theoretical concepts were explored, building upon fundamental principles with sophisticated extensions."
]

conclusion_phrases = [
    "The session concluded with practical exercises and problem-solving activities to reinforce understanding.",
    "Comprehensive examples and applications were provided to demonstrate real-world relevance and utility.",
    "The lecture ended with discussion of current research developments and future applications in the field.",
    "Extensive Q&A session addressed common misconceptions and clarified complex theoretical points.",
    "Hands-on demonstrations and interactive elements helped solidify conceptual understanding.",
    "The class concluded with assignment of practice problems and recommended additional reading materials.",
    "Summary discussions reinforced key concepts and highlighted connections to previously covered material.",
    "The session ended with exploration of advanced topics and suggestions for further independent study.",
    "Practical implementation examples were provided with step-by-step guidance for applying learned concepts.",
    "The lecture concluded with examination of current industry applications and emerging technological trends."
]

reminder_days = [2, 3, 4, 5, 6, 7, 8]

def generate_note(subject, topic):
    """Generate a lengthy academic note"""
    intro = random.choice(intro_phrases).format(topic)

    # Build the main content
    expansions = random.sample(content_expansions, random.randint(3, 5))
    descriptions = random.sample(detailed_descriptions, random.randint(2, 4))
    conclusion = random.choice(conclusion_phrases)

    # Combine all parts
    content_parts = [intro] + expansions + descriptions + [conclusion]

    # Add connecting words and make it flow better
    connectors = [" We explored ", " The discussion included ", " Additionally, we examined ",
                 " Furthermore, the lecture covered ", " We also analyzed ", " The session addressed ",
                 " In detail, we studied ", " Comprehensive coverage included ", " We investigated "]

    note = content_parts[0]
    for i, part in enumerate(content_parts[1:-1], 1):
        connector = random.choice(connectors)
        note += connector + part.lower()

    note += " " + content_parts[-1]

    return note

def generate_summary(subject, topic, reminder_days):
    """Generate a concise summary with detailed revision reminders"""
    key_concepts = [
        f"Key concepts of {topic} explained",
        f"{topic.title()} fundamentals covered",
        f"Comprehensive {topic} analysis provided",
        f"{topic.title()} principles and applications discussed",
        f"Detailed {topic} methodology presented",
        f"{topic.title()} theory and practice examined"
    ]

    focuses = [
        "with focus on fundamentals and applications",
        "including theoretical foundations and practical implementations",
        "covering key principles and real-world examples",
        "with emphasis on methodology and applications",
        "including comprehensive analysis and case studies",
        "focusing on core concepts and practical usage"
    ]

    # Generate specific revision concepts based on the topic
    revision_concepts = generate_revision_concepts(subject, topic)

    concept = random.choice(key_concepts)
    focus = random.choice(focuses)
    days = random.choice(reminder_days)

    return f"Summary: {concept} {focus}. Subject: {subject}; Reminder: Revise {days} days before exam - Review concepts: {revision_concepts}"

def generate_revision_concepts(subject, topic):
    """Generate specific concepts to revise based on subject and topic"""

    concept_mappings = {
        "Machine Learning": {
            "gradient descent": ["cost functions", "learning rates", "convergence criteria", "optimization algorithms"],
            "neural networks": ["activation functions", "backpropagation", "weight initialization", "network architectures"],
            "support vector machines": ["kernel functions", "margin maximization", "regularization parameters", "classification boundaries"],
            "decision trees": ["splitting criteria", "pruning methods", "entropy calculations", "tree depth optimization"],
            "random forests": ["ensemble methods", "bagging techniques", "feature importance", "out-of-bag error"],
            "k-means clustering": ["centroid calculations", "cluster initialization", "distance metrics", "convergence conditions"],
            "linear regression": ["least squares method", "coefficient interpretation", "residual analysis", "assumption validation"],
            "logistic regression": ["sigmoid function", "maximum likelihood estimation", "odds ratios", "classification metrics"],
            "reinforcement learning": ["reward functions", "Q-learning", "policy gradients", "exploration vs exploitation"],
            "deep learning": ["deep architectures", "vanishing gradients", "regularization techniques", "transfer learning"]
        },
        "Deep Learning": {
            "backpropagation": ["chain rule", "gradient computation", "weight updates", "error propagation"],
            "activation functions": ["ReLU properties", "sigmoid limitations", "tanh characteristics", "function derivatives"],
            "CNN architectures": ["convolutional layers", "pooling operations", "feature maps", "spatial hierarchies"],
            "RNN structures": ["sequential processing", "hidden states", "vanishing gradients", "memory mechanisms"],
            "transformer models": ["attention mechanisms", "positional encoding", "multi-head attention", "encoder-decoder"],
            "attention mechanisms": ["query-key-value", "self-attention", "cross-attention", "attention weights"],
            "BERT models": ["bidirectional encoding", "masked language modeling", "next sentence prediction", "fine-tuning"],
            "GPT architecture": ["autoregressive generation", "transformer blocks", "causal masking", "text completion"],
            "autoencoders": ["encoding-decoding", "latent representations", "reconstruction loss", "dimensionality reduction"],
            "GANs": ["generator networks", "discriminator training", "adversarial loss", "mode collapse"]
        },
        "Biology": {
            "photosynthesis": ["light reactions", "Calvin cycle", "chloroplast structure", "electron transport chain"],
            "cellular respiration": ["glycolysis", "Krebs cycle", "electron transport", "ATP synthesis"],
            "DNA replication": ["DNA polymerase", "Okazaki fragments", "proofreading mechanisms", "replication fork"],
            "protein synthesis": ["transcription", "translation", "ribosomes", "genetic code"],
            "mitosis and meiosis": ["chromosome separation", "crossing over", "genetic diversity", "cell division phases"],
            "genetics": ["Mendelian inheritance", "gene expression", "mutations", "genetic variation"],
            "evolution": ["natural selection", "genetic drift", "speciation", "phylogenetic relationships"],
            "ecology": ["ecosystem dynamics", "population growth", "species interactions", "environmental factors"],
            "enzyme kinetics": ["Michaelis-Menten", "competitive inhibition", "allosteric regulation", "enzyme specificity"],
            "cell membrane structure": ["phospholipid bilayer", "membrane proteins", "transport mechanisms", "membrane fluidity"]
        },
        "Chemistry": {
            "organic reactions": ["reaction mechanisms", "stereochemistry", "functional groups", "reaction conditions"],
            "acid-base chemistry": ["pH calculations", "buffer systems", "titration curves", "acid-base equilibria"],
            "electrochemistry": ["redox reactions", "electrode potentials", "galvanic cells", "electrolysis"],
            "thermodynamics": ["enthalpy", "entropy", "Gibbs free energy", "equilibrium constants"],
            "kinetics": ["reaction rates", "rate laws", "activation energy", "catalysis"],
            "equilibrium": ["Le Chatelier's principle", "equilibrium constants", "reaction quotients", "dynamic equilibrium"],
            "molecular orbital theory": ["orbital overlap", "bonding/antibonding", "molecular geometry", "hybridization"],
            "spectroscopy": ["NMR analysis", "IR spectroscopy", "mass spectrometry", "UV-visible spectroscopy"],
            "crystallography": ["crystal structures", "unit cells", "X-ray diffraction", "lattice parameters"],
            "polymer chemistry": ["polymerization", "polymer properties", "molecular weight", "polymer characterization"]
        },
        "Physics": {
            "electromagnetism": ["electric fields", "magnetic fields", "Maxwell equations", "electromagnetic induction"],
            "quantum mechanics": ["wave functions", "Schrödinger equation", "quantum operators", "measurement theory"],
            "thermodynamics": ["laws of thermodynamics", "heat engines", "entropy", "phase transitions"],
            "optics": ["wave interference", "diffraction", "polarization", "optical instruments"],
            "mechanics": ["Newton's laws", "conservation principles", "rotational dynamics", "oscillations"],
            "wave physics": ["wave equations", "standing waves", "wave interference", "Doppler effect"],
            "nuclear physics": ["radioactive decay", "nuclear reactions", "binding energy", "nuclear models"],
            "particle physics": ["standard model", "fundamental forces", "particle interactions", "conservation laws"],
            "relativity": ["special relativity", "time dilation", "length contraction", "mass-energy equivalence"],
            "condensed matter": ["crystal structures", "electronic properties", "phase transitions", "collective phenomena"]
        },
        "Mathematics": {
            "calculus": ["derivatives", "integrals", "limits", "fundamental theorem"],
            "linear algebra": ["matrices", "eigenvalues", "vector spaces", "linear transformations"],
            "differential equations": ["ordinary DEs", "partial DEs", "solution methods", "boundary conditions"],
            "probability": ["probability distributions", "random variables", "expectation", "variance"],
            "statistics": ["hypothesis testing", "confidence intervals", "regression analysis", "statistical inference"],
            "number theory": ["prime numbers", "modular arithmetic", "Diophantine equations", "cryptographic applications"],
            "abstract algebra": ["groups", "rings", "fields", "homomorphisms"],
            "real analysis": ["sequences", "series", "continuity", "convergence"],
            "complex analysis": ["analytic functions", "complex integration", "residue theory", "conformal mapping"],
            "topology": ["topological spaces", "continuity", "compactness", "connectedness"]
        },
        "Computer Science": {
            "algorithms": ["time complexity", "space complexity", "algorithm design", "optimization techniques"],
            "data structures": ["arrays", "linked lists", "trees", "hash tables"],
            "database systems": ["normalization", "SQL queries", "indexing", "transaction management"],
            "software engineering": ["design patterns", "testing methods", "version control", "project management"],
            "computer networks": ["protocol layers", "routing algorithms", "network security", "performance analysis"],
            "operating systems": ["process management", "memory management", "file systems", "synchronization"],
            "computer graphics": ["3D transformations", "rendering algorithms", "lighting models", "texture mapping"],
            "artificial intelligence": ["search algorithms", "knowledge representation", "machine reasoning", "AI ethics"],
            "cybersecurity": ["encryption methods", "threat analysis", "security protocols", "vulnerability assessment"],
            "distributed systems": ["consistency models", "fault tolerance", "distributed algorithms", "scalability"]
        },
        "Economics": {
            "macroeconomics": ["GDP measurement", "inflation", "unemployment", "monetary policy"],
            "microeconomics": ["supply and demand", "market structures", "consumer theory", "producer theory"],
            "econometrics": ["regression analysis", "time series", "panel data", "causal inference"],
            "game theory": ["Nash equilibrium", "strategic interactions", "auction theory", "cooperative games"],
            "behavioral economics": ["cognitive biases", "prospect theory", "bounded rationality", "experimental methods"],
            "international trade": ["comparative advantage", "trade barriers", "exchange rates", "trade agreements"],
            "monetary policy": ["interest rates", "money supply", "central banking", "inflation targeting"],
            "fiscal policy": ["government spending", "taxation", "budget deficits", "fiscal multipliers"],
            "market structures": ["perfect competition", "monopoly", "oligopoly", "market power"],
            "economic growth": ["growth models", "productivity", "human capital", "technological progress"]
        },
        "Psychology": {
            "cognitive psychology": ["memory processes", "attention", "perception", "decision making"],
            "behavioral psychology": ["conditioning", "reinforcement", "behavior modification", "learning theories"],
            "developmental psychology": ["cognitive development", "social development", "attachment theory", "developmental stages"],
            "social psychology": ["group dynamics", "social cognition", "attitudes", "interpersonal relationships"],
            "abnormal psychology": ["psychological disorders", "diagnostic criteria", "treatment approaches", "mental health"],
            "neuropsychology": ["brain-behavior relationships", "neurological assessment", "cognitive functions", "brain imaging"],
            "research methods": ["experimental design", "statistical analysis", "sampling methods", "ethical considerations"],
            "statistics in psychology": ["descriptive statistics", "inferential statistics", "effect sizes", "statistical significance"],
            "personality psychology": ["personality theories", "trait models", "assessment methods", "individual differences"],
            "learning theories": ["classical conditioning", "operant conditioning", "social learning", "cognitive learning"]
        },
        "Statistics": {
            "descriptive statistics": ["measures of central tendency", "measures of variability", "data visualization", "summary statistics"],
            "inferential statistics": ["hypothesis testing", "confidence intervals", "p-values", "statistical significance"],
            "hypothesis testing": ["null hypothesis", "alternative hypothesis", "Type I/II errors", "power analysis"],
            "regression analysis": ["linear regression", "multiple regression", "model assumptions", "residual analysis"],
            "ANOVA": ["analysis of variance", "F-tests", "multiple comparisons", "factorial designs"],
            "experimental design": ["randomization", "control groups", "blocking", "factorial experiments"],
            "sampling methods": ["random sampling", "stratified sampling", "cluster sampling", "sampling distributions"],
            "probability distributions": ["normal distribution", "binomial distribution", "Poisson distribution", "continuous distributions"],
            "Bayesian statistics": ["prior distributions", "posterior distributions", "Bayes' theorem", "Bayesian inference"],
            "time series analysis": ["trend analysis", "seasonal decomposition", "autocorrelation", "forecasting methods"]
        }
    }

    # Get concepts for this subject-topic combination
    subject_concepts = concept_mappings.get(subject, {})
    topic_concepts = subject_concepts.get(topic, ["fundamental principles", "key applications", "theoretical foundations", "practical methods"])

    # Select 2-4 concepts randomly
    selected_concepts = random.sample(topic_concepts, min(random.randint(2, 4), len(topic_concepts)))

    return ", ".join(selected_concepts)

def generate_dataset(num_entries=10000):
    """Generate the complete dataset"""
    notes = []
    summaries = []

    print(f"Generating {num_entries} entries...")

    for i in range(num_entries):
        if i % 1000 == 0:
            print(f"Generated {i} entries...")

        # Select random subject and topic
        subject = random.choice(list(subjects_topics.keys()))
        topic = random.choice(subjects_topics[subject])

        # Generate note and summary
        note = generate_note(subject, topic)
        summary = generate_summary(subject, topic, reminder_days)

        notes.append(note)
        summaries.append(summary)

    # Create DataFrame
    df = pd.DataFrame({
        'notes': notes,
        'summary': summaries
    })

    return df

# Generate the dataset
print("Starting dataset generation...")
dataset = generate_dataset(10000)

# Save to CSV
dataset.to_csv('academic_notes_10k.csv', index=False)
print(f"Dataset saved! Shape: {dataset.shape}")
print("\nFirst few examples:")
for i in range(3):
    print(f"\nExample {i+1}:")
    print(f"Note length: {len(dataset.iloc[i]['notes'])} characters")
    print(f"Note: {dataset.iloc[i]['notes'][:200]}...")
    print(f"Summary: {dataset.iloc[i]['summary']}")

print(f"\nDataset statistics:")
print(f"Average note length: {dataset['notes'].str.len().mean():.0f} characters")
print(f"Average summary length: {dataset['summary'].str.len().mean():.0f} characters")
print(f"Subject distribution:")
print(dataset['summary'].str.extract(r'Subject: ([^;]+)')[0].value_counts())

Starting dataset generation...
Generating 10000 entries...
Generated 0 entries...
Generated 1000 entries...
Generated 2000 entries...
Generated 3000 entries...
Generated 4000 entries...
Generated 5000 entries...
Generated 6000 entries...
Generated 7000 entries...
Generated 8000 entries...
Generated 9000 entries...
Dataset saved! Shape: (10000, 2)

First few examples:

Example 1:
Note length: 786 characters
Note: Today's lecture on wave physics provided a comprehensive examination of The session addressed mathematical foundations and theoretical principles The discussion included advanced concepts and cutting-...
Summary: Summary: Wave Physics theory and practice examined covering key principles and real-world examples. Subject: Physics; Reminder: Revise 2 days before exam - Review concepts: wave equations, standing waves

Example 2:
Note length: 1044 characters
Note: Today's detailed discussion of protein synthesis covered multiple aspects including The session addressed experimental p

In [8]:
# ========================================
# DATASET MAPPING FOR FINE-TUNING
# ========================================

import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
import torch

# 1. Load your dataset
# Use the dataset generated in the previous cell, which is already in the environment
df = pd.read_csv('/content/academic_notes_10k.csv')  # Replace with your file path
print(f"Original dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")

# 2. Display sample data
print(f"\nSample data:")
print(f"Notes: {df.iloc[0]['notes'][:150]}...")
print(f"Summary: {df.iloc[0]['summary']}")
# print(f"Remainder: {df.iloc[0]['remainder']}") # Removed as 'remainder' column does not exist

# 3. Data cleaning and preparation
# Remove any null values
df = df.dropna()
print(f"\nAfter removing nulls: {df.shape}")

# Convert to string type to avoid issues
df['notes'] = df['notes'].astype(str)
df['summary'] = df['summary'].astype(str)
# df['remainder'] = df['remainder'].astype(str) # Removed as 'remainder' column does not exist

# 4. Create target (using only the 'summary' column)
df['target'] = df['summary']
print(f"\nTarget example:")
print(f"Target: {df.iloc[0]['target']}")


# 5. Check target lengths
target_lengths = df['target'].str.len()
print(f"\nTarget length statistics:")
print(f"Mean: {target_lengths.mean():.0f} characters")
print(f"Max: {target_lengths.max()} characters")
print(f"Min: {target_lengths.min()} characters")

# 6. Load tokenizer for length checking
tokenizer = AutoTokenizer.from_pretrained('t5-small')

# 7. Define preprocessing function
def preprocess_function(examples):
    """
    Map the dataset for T5 model training
    Input: notes -> Output: target (summary)
    """
    # Add task prefix for T5 models
    inputs = []
    for note in examples['notes']:
        # You can customize the prefix based on your task
        inputs.append(f"summarize: {note}")

    # Tokenize inputs (notes)
    model_inputs = tokenizer(
        inputs,
        max_length=512,          # Max input length
        truncation=True,         # Truncate if longer
        padding='max_length',    # Pad to max length
        return_tensors='pt'
    )

    # Tokenize targets (summary)
    targets = tokenizer(
        examples['target'],
        max_length=256,          # Max target length
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

    # Set labels for training
    model_inputs['labels'] = targets['input_ids']

    # Replace padding token id's of the labels by -100 so they are ignored by the loss function
    model_inputs['labels'] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in model_inputs['labels']
    ]

    return model_inputs

# 8. Split dataset
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"\nDataset split:")
print(f"Training: {len(train_df)} samples")
print(f"Validation: {len(val_df)} samples")
print(f"Test: {len(test_df)} samples")

# 9. Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df[['notes', 'target']])
val_dataset = Dataset.from_pandas(val_df[['notes', 'target']])
test_dataset = Dataset.from_pandas(test_df[['notes', 'target']])

print(f"\nDataset objects created:")
print(f"Train dataset: {train_dataset}")
print(f"Validation dataset: {val_dataset}")
print(f"Test dataset: {test_dataset}")

# 10. Apply preprocessing/mapping
print(f"\nApplying preprocessing...")

# Map the preprocessing function to datasets
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=100,  # Process in batches for efficiency
    remove_columns=['notes', 'target']  # Remove original columns
)

tokenized_val = val_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=100,
    remove_columns=['notes', 'target']
)

tokenized_test = test_dataset.map(
    preprocess_function,
    batched=True,
    batch_size=100,
    remove_columns=['notes', 'target']
)

print(f"Preprocessing complete!")
print(f"Tokenized train dataset: {tokenized_train}")

# 11. Verify mapping worked correctly
print(f"\nVerifying mapping...")
sample = tokenized_train[0]
print(f"Sample keys: {sample.keys()}")
print(f"Input shape: {sample['input_ids'].shape if hasattr(sample['input_ids'], 'shape') else len(sample['input_ids'])}")
print(f"Labels shape: {sample['labels'].shape if hasattr(sample['labels'], 'shape') else len(sample['labels'])}")

# Decode to verify content
decoded_input = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
decoded_labels = tokenizer.decode([l for l in sample['labels'] if l != -100], skip_special_tokens=True)

print(f"\nDecoded sample:")
print(f"Input: {decoded_input[:100]}...")
print(f"Label: {decoded_labels[:100]}...")

# 12. Check token lengths
def analyze_lengths(dataset, name):
    input_lengths = [len([t for t in example['input_ids'] if t != tokenizer.pad_token_id]) for example in dataset]
    label_lengths = [len([l for l in example['labels'] if l != -100]) for example in dataset]

    print(f"\n{name} set token length analysis:")
    print(f"Input tokens - Mean: {sum(input_lengths)/len(input_lengths):.1f}, Max: {max(input_lengths)}")
    print(f"Label tokens - Mean: {sum(label_lengths)/len(label_lengths):.1f}, Max: {max(label_lengths)}")

analyze_lengths(tokenized_train, "Train")
analyze_lengths(tokenized_val, "Validation")

# 13. Save processed datasets (optional)
print(f"\nSaving processed datasets...")
tokenized_train.save_to_disk("./processed_data/train")
tokenized_val.save_to_disk("./processed_data/val")
tokenized_test.save_to_disk("./processed_data/test")

print(f"✅ Dataset mapping complete!")
print(f"📊 Summary:")
print(f"  - Original dataset: {len(df)} samples")
print(f"  - Training: {len(tokenized_train)} samples")
print(f"  - Validation: {len(tokenized_val)} samples")
print(f"  - Test: {len(tokenized_test)} samples")
print(f"  - Input format: 'summarize: [notes]'")
print(f"  - Target format: '[summary]'")
print(f"  - Ready for fine-tuning!")

# Return the tokenized datasets for training
print(f"\nUse these variables for training:")
print(f"- tokenized_train")
print(f"- tokenized_val")
print(f"- tokenized_test")

Original dataset shape: (10000, 2)
Columns: ['notes', 'summary']

Sample data:
Notes: Today's lecture on wave physics provided a comprehensive examination of The session addressed mathematical foundations and theoretical principles The ...
Summary: Summary: Wave Physics theory and practice examined covering key principles and real-world examples. Subject: Physics; Reminder: Revise 2 days before exam - Review concepts: wave equations, standing waves

After removing nulls: (10000, 2)

Target example:
Target: Summary: Wave Physics theory and practice examined covering key principles and real-world examples. Subject: Physics; Reminder: Revise 2 days before exam - Review concepts: wave equations, standing waves

Target length statistics:
Mean: 242 characters
Max: 302 characters
Min: 182 characters


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]


Dataset split:
Training: 8000 samples
Validation: 1000 samples
Test: 1000 samples

Dataset objects created:
Train dataset: Dataset({
    features: ['notes', 'target', '__index_level_0__'],
    num_rows: 8000
})
Validation dataset: Dataset({
    features: ['notes', 'target', '__index_level_0__'],
    num_rows: 1000
})
Test dataset: Dataset({
    features: ['notes', 'target', '__index_level_0__'],
    num_rows: 1000
})

Applying preprocessing...


...

Preprocessing complete!
Tokenized train dataset: Dataset({
    features: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8000
})

Verifying mapping...
Sample keys: dict_keys(['__index_level_0__', 'input_ids', 'attention_mask', 'labels'])
Input shape: 512
Labels shape: 256

Decoded sample:
Input: summarize: Today's lecture on sequence modeling provided a comprehensive examination of We explored ...
Label: Summary: Sequence Modeling theory and practice examined including theoretical foundations and practi...

Train set token length analysis:
Input tokens - Mean: 131.3, Max: 183
Label tokens - Mean: 46.7, Max: 64

Validation set token length analysis:
Input tokens - Mean: 130.6, Max: 176
Label tokens - Mean: 46.6, Max: 60

Saving processed datasets...


Saving the dataset (0/1 shards):   0%|          | 0/8000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

✅ Dataset mapping complete!
📊 Summary:
  - Original dataset: 10000 samples
  - Training: 8000 samples
  - Validation: 1000 samples
  - Test: 1000 samples
  - Input format: 'summarize: [notes]'
  - Target format: '[summary]'
  - Ready for fine-tuning!

Use these variables for training:
- tokenized_train
- tokenized_val
- tokenized_test


In [9]:
!pip install transformers datasets evaluate rouge-score tensorboard accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=057661ef883931dee4b35468431083d39a76128d9dd46171139ac41702b39c53
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [ ]:
# Alternative: Use automatic mixed precision settings
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=2,                    # Reduce epochs for faster testing
    per_device_train_batch_size=1,         # Even smaller batch size
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,        # Maintain effective batch size
    learning_rate=5e-4,                    # Slightly higher LR for smaller batches
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=1000,                       # Less frequent evaluation
    save_steps=1000,
    logging_steps=100,
    save_total_limit=2,

    # Safe precision settings
    fp16=False,
    bf16=False,
    # tf32=True if torch.cuda.is_available() else False,  # Removed as it requires Ampere or newer GPU

    # Memory optimizations
    gradient_checkpointing=True,           # Save memory
    dataloader_pin_memory=False,
    dataloader_num_workers=0,
    remove_unused_columns=True,
    seed=42,
)

In [4]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model
import evaluate
import numpy as np

# ---------------------------
# 1. Config
# ---------------------------
MODEL_NAME = "google/flan-t5-base"  # smaller model for Colab
OUTPUT_DIR = "./flan-t5-base-lora"
MAX_INPUT_LENGTH = 256   # reduced for Colab
MAX_TARGET_LENGTH = 64
EPOCHS = 1

# ---------------------------
# 2. Load dataset (CSV)
# ---------------------------
# Make sure your CSV has columns: "notes" (input) and "summary" (target)
dataset = load_dataset("csv", data_files={"data": "/content/academic_notes_10k.csv"})["data"]

# Split into train/validation (90/10)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_data = dataset["train"]
val_data = dataset["test"]

print("Sample row:", train_data[0])

# ---------------------------
# 3. Tokenizer
# ---------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["notes"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)
    labels = tokenizer(examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
tokenized_val = val_data.map(preprocess_function, batched=True, remove_columns=val_data.column_names)

# ---------------------------
# 4. Load model in fp16 (no bitsandbytes)
# ---------------------------
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.float16,  # use half precision
)

# Enable gradient checkpointing (saves VRAM)
model.gradient_checkpointing_enable()

# Add LoRA adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],  # LoRA on attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# ---------------------------
# 5. Data collator
# ---------------------------
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ---------------------------
# 6. Metrics
# ---------------------------
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}

# ---------------------------
# 7. Training args
# ---------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=1,     # small batch size
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,    # simulate larger batch
    learning_rate=2e-4,
    logging_steps=50,
    eval_strategy="steps", # Corrected from evaluation_strategy
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="tensorboard",
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
)

# ---------------------------
# 8. Trainer
# ---------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ---------------------------
# 9. Train
# ---------------------------
train_result = trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("✅ Training finished!")

# ---------------------------
# 10. Test inference
# ---------------------------
def generate_summary(text, max_length=64):
    inputs = tokenizer("summarize: " + text, return_tensors="pt", truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\n🔍 Sample test:")
sample_text = val_data[0]["notes"]
print("Input:", sample_text[:200])
print("Reference:", val_data[0]["summary"])
print("Generated:", generate_summary(sample_text))

Sample row: {'notes': "Today's lecture on topology provided a comprehensive examination of The session addressed interdisciplinary connections and applications In detail, we studied problem-solving strategies and methodological approaches The session addressed technical specifications and implementation details The discussion included error analysis and troubleshooting methods We investigated performance evaluation metrics were introduced with benchmarking strategies and comparative assessment methods. Furthermore, the lecture covered current research trends and future directions were explored, including emerging technologies and methodological innovations. The session addressed advanced theoretical concepts were explored, building upon fundamental principles with sophisticated extensions. The discussion included real-world case studies demonstrated practical applications across various industries and research domains. Comprehensive examples and applications were provided to demonstrat

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096


/tmp/ipython-input-1099308227.py:119: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.76 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.76 GiB is free. Process 27097 has 11.98 GiB memory in use. Of the allocated memory 10.40 GiB is allocated by PyTorch, and 1.45 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:
# Reinstall the latest bitsandbytes
!pip install -U bitsandbytes

# Then, load the model with 4-bit quantization from cell -rPPU_XQXdJw
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch

MODEL_NAME = "google/flan-t5-base"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

# Re-apply LoRA configuration
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)
model = get_peft_model(model, lora_config)

# Print trainable parameters info
model.print_trainable_parameters()

# Then, you can rerun the training cell (NKYD2R5jLQe5)
print("Bitsandbytes reinstalled and model loaded with 4-bit quantization. Please rerun the training cell (NKYD2R5jLQe5).")

trainable params: 1,769,472 || all params: 249,347,328 || trainable%: 0.7096
Bitsandbytes reinstalled and model loaded with 4-bit quantization. Please rerun the training cell (NKYD2R5jLQe5).


In [ ]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes==0.43.1
!pip install -U transformers accelerate peft datasets evaluate


Found existing installation: bitsandbytes 0.47.0
Uninstalling bitsandbytes-0.47.0:
  Successfully uninstalled bitsandbytes-0.47.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.6 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft
!pip install -U datasets evaluate


In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForSeq2SeqLM, AutoTokenizer

MODEL_NAME = "google/flan-t5-base"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
!pip install -U transformers

# then Runtime -> Restart runtime in Colab


In [ ]:
# ========================================
# STEP 1: INSTALL REQUIRED PACKAGES
# ========================================
# Run this cell first!

!pip install -q bitsandbytes>=0.41.0
!pip install -q peft>=0.5.0
!pip install -q transformers>=4.34.0
!pip install -q datasets>=2.14.0
!pip install -q evaluate>=0.4.0
!pip install -q rouge-score
!pip install -q accelerate>=0.24.0

print("✅ All packages installed!")

# ========================================
# STEP 2: RESTART RUNTIME
# ========================================
# After running the above, go to Runtime -> Restart Runtime
# Then run the code below

# ========================================
# STEP 3: LORA FINE-TUNING CODE
# ========================================

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model

import evaluate
import numpy as np
import os

# Set environment variable to help with memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("🚀 Starting LoRA Fine-tuning Setup")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# ---------------------------
# 1. Config
# ---------------------------
MODEL_NAME = "google/flan-t5-base"  # smaller model for Colab
OUTPUT_DIR = "./flan-t5-base-lora"
MAX_INPUT_LENGTH = 256   # reduced for Colab
MAX_TARGET_LENGTH = 64
EPOCHS = 2

print(f"Model: {MODEL_NAME}")
print(f"Output directory: {OUTPUT_DIR}")

# ---------------------------
# 2. Load dataset (from your CSV)
# ---------------------------
print("📊 Loading dataset...")

# Load your preprocessed dataset
dataset = load_dataset("csv", data_files={"data": "/content/academic_notes_10k.csv"})["data"]

# Check dataset structure
print(f"Dataset columns: {dataset.column_names}")
print(f"Dataset size: {len(dataset)}")
print("Sample row:", dataset[0])

# Split into train/validation (90/10)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_data = dataset["train"]
val_data = dataset["test"]

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

# ---------------------------
# 3. Tokenizer & Preprocessing
# ---------------------------
print("🔧 Setting up tokenizer...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    # Make sure to use the correct column name from your dataset
    inputs = ["summarize: " + doc for doc in examples["notes"]]
    model_inputs = tokenizer(inputs, max_length=MAX_INPUT_LENGTH, truncation=True)
    labels = tokenizer(examples["summary"], max_length=MAX_TARGET_LENGTH, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

print("📝 Preprocessing datasets...")
tokenized_train = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
tokenized_val = val_data.map(preprocess_function, batched=True, remove_columns=val_data.column_names)

print(f"Tokenized train dataset: {tokenized_train}")
print(f"Tokenized validation dataset: {tokenized_val}")

# ---------------------------
# 4. Model with Quantization + LoRA
# ---------------------------
print("🤖 Loading model with 4-bit quantization...")

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

# Load model with quantization
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

print("✅ Model loaded with 4-bit quantization")

# Enable gradient checkpointing (saves VRAM)
model.gradient_checkpointing_enable()

# Configure LoRA
print("🎯 Setting up LoRA configuration...")
lora_config = LoraConfig(
    r=16,                           # LoRA rank
    lora_alpha=32,                  # LoRA scaling parameter
    target_modules=["q", "v"],      # Apply LoRA to query and value projections
    lora_dropout=0.05,              # LoRA dropout
    bias="none",                    # Don't train bias parameters
    task_type="SEQ_2_SEQ_LM"       # Task type
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Enable gradient calculation for inputs (required for LoRA with gradient checkpointing)
model.enable_input_require_grads()

# Print trainable parameters info
model.print_trainable_parameters()

# ---------------------------
# 5. Data collator
# ---------------------------
print("📦 Setting up data collator...")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ---------------------------
# 6. Evaluation metrics
# ---------------------------
print("📏 Loading evaluation metrics...")
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v, 4) for k, v in result.items()}

# ---------------------------
# 7. Training arguments
# ---------------------------
print("⚙️ Setting up training arguments...")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=2,         # Small batch size for Colab
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,         # Effective batch size = 2 * 8 = 16
    learning_rate=2e-4,                    # Good for LoRA
    weight_decay=0.01,
    warmup_ratio=0.1,

    # Evaluation and saving
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",

    # Logging
    logging_steps=50,
    logging_dir=f"{OUTPUT_DIR}/logs",
    report_to=["tensorboard"],

    # Performance
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=True,
    remove_unused_columns=True,
    push_to_hub=False,
    seed=42,
)

print("✅ Training arguments configured")

# ---------------------------
# 8. Initialize Trainer
# ---------------------------
print("🏋️ Initializing trainer...")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("✅ Trainer initialized successfully")

# ---------------------------
# 9. Start Training
# ---------------------------
print("🎯 Starting LoRA fine-tuning...")
print("This will be much faster than full fine-tuning!")

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Start training
train_result = trainer.train()

print("✅ Training completed!")
print(f"Training results: {train_result.metrics}")

# ---------------------------
# 10. Save the model
# ---------------------------
print("💾 Saving the fine-tuned model...")

trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Model saved to: {OUTPUT_DIR}")

# ---------------------------
# 11. Test inference
# ---------------------------
print("🧪 Testing the fine-tuned model...")

def generate_summary(text, max_length=64):
    """Generate summary using the fine-tuned model"""
    inputs = tokenizer(
        "summarize: " + text,
        return_tensors="pt",
        max_length=MAX_INPUT_LENGTH,
        truncation=True
    ).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,
            length_penalty=0.6,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with a sample
print("\n📝 Sample test:")
sample_text = val_data[0]["notes"]
reference_summary = val_data[0]["summary"]

print(f"Input (first 200 chars): {sample_text[:200]}...")
print(f"Reference summary: {reference_summary}")

generated_summary = generate_summary(sample_text)
print(f"Generated summary: {generated_summary}")

# ---------------------------
# 12. Final evaluation
# ---------------------------
print("\n📊 Running final evaluation...")
final_results = trainer.evaluate()

print("Final evaluation results:")
for key, value in final_results.items():
    print(f"  {key}: {value}")

print("\n🎉 LoRA fine-tuning completed successfully!")
print(f"📁 Model saved in: {OUTPUT_DIR}")
print(f"📊 Tensorboard logs: {OUTPUT_DIR}/logs")
print("✨ Your model is ready to use!")

# Clean up GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("🧹 GPU memory cleaned")

✅ All packages installed!
🚀 Starting LoRA Fine-tuning Setup
CUDA available: False
Model: google/flan-t5-base
Output directory: ./flan-t5-base-lora
📊 Loading dataset...
Dataset columns: ['notes', 'summary']
Dataset size: 10000
Sample row: {'notes': 'This advanced lecture on abstract algebra examined various components of Furthermore, the lecture covered problem-solving strategies and methodological approaches We also analyzed mathematical foundations and theoretical principles Additionally, we examined practical applications and real-world implementations We explored optimization techniques and efficiency considerations We also analyzed performance evaluation metrics were introduced with benchmarking strategies and comparative assessment methods. Furthermore, the lecture covered advanced theoretical concepts were explored, building upon fundamental principles with sophisticated extensions. In detail, we studied problem-solving frameworks were established with systematic approaches to ta

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📝 Preprocessing datasets...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenized train dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9000
})
Tokenized validation dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})
🤖 Loading model with 4-bit quantization...


ImportError: The installed version of bitsandbytes (<0.43.1) requires CUDA, but CUDA is not available. You may need to install PyTorch with CUDA support or upgrade bitsandbytes to >=0.43.1.

In [ ]:
# Run this first - force reinstall everything
!pip uninstall -y bitsandbytes transformers peft
!pip install bitsandbytes==0.41.1 --force-reinstall
!pip install transformers==4.35.0 --force-reinstall
!pip install peft==0.6.0 --force-reinstall
!pip install accelerate==0.24.1 --force-reinstall

# Check if it worked
import bitsandbytes
print(f"✅ bitsandbytes version: {bitsandbytes.__version__}")

Found existing installation: bitsandbytes 0.41.1
Uninstalling bitsandbytes-0.41.1:
  Successfully uninstalled bitsandbytes-0.41.1
Found existing installation: transformers 4.35.0
Uninstalling transformers-4.35.0:
  Successfully uninstalled transformers-4.35.0
  Using cached bitsandbytes-0.41.1-py3-none-any.whl.metadata (9.8 kB)
Using cached bitsandbytes-0.41.1-py3-none-any.whl (92.6 MB)
  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.34.5-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached regex-2025.9.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)


  Using cached peft-0.6.0-py3-none-any.whl.metadata (23 kB)
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached psutil-7.0.0-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached torch-2.8.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached transformers-4.56.1-py3-none-any.whl.metadata (42 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached accelerate-1.10.1-py3-none-any.whl.metadata (19 kB)
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached huggingface_hub-0.34.5-py3-none-any.whl.metadata (14 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)


ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft datasets evaluate


In [ ]:
# Install required packages

!pip install -U accelerate transformers peft

!pip install -U bitsandbytes
!pip install transformers datasets evaluate rouge-score tensorboard

Building AI Agent

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

MODEL_PATH = "./flan-t5-base-lora"  # your fine-tuned model dir

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

# Create a pipeline for text2text
summarizer = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


In [ ]:
from transformers import Tool, HfAgent

# Wrap your summarizer pipeline as a tool
summarizer_tool = Tool(
    name="custom_summarizer",
    func=lambda text: summarizer("summarize: " + text, max_length=64)[0]["generated_text"],
    description="Summarizes academic notes into concise text"
)

# Create an agent with your tool
agent = HfAgent(tools=[summarizer_tool])

# Ask the agent to perform a task
print(agent.run("Summarize this note: Deep learning models require a lot of data and compute to train effectively."))


In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

# Wrap pipeline in LangChain LLM
llm = HuggingFacePipeline(pipeline=summarizer)

# Define a tool for summarization
tools = [
    Tool(
        name="Summarizer",
        func=lambda q: llm("summarize: " + q),
        description="Summarize long academic notes into short summaries"
    )
]

# Build the agent
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Run the agent
agent.run("Please summarize the following note: Attention is all you need introduces Transformers architecture.")


#Train Agent

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

MODEL_PATH = "./flan-t5-base-lora"  # path to fine-tuned model

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

summarizer = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.agents import initialize_agent, Tool, AgentType

# Wrap summarizer into LangChain LLM
llm = HuggingFacePipeline(pipeline=summarizer)

# Define tool
tools = [
    Tool(
        name="Summarizer",
        func=lambda q: llm("summarize: " + q),
        description="Summarizes academic notes into short summaries."
    )
]

# Initialize agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [ ]:
query = "Summarize this note: Transformers allow parallel processing unlike RNNs."
response = agent.run(query)
print("🔍 Agent response:", response)


#Evalaute your agent using suitable metrics

In [ ]:
import evaluate
import numpy as np

# Metrics
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

def evaluate_agent(agent, val_data, num_samples=200):
    preds, refs = [], []

    for i in range(min(num_samples, len(val_data))):
        input_text = val_data[i]["notes"]
        reference = val_data[i]["summary"]

        # Run through agent
        try:
            output = agent.run("Summarize this note: " + input_text)
        except:
            output = ""

        preds.append(output.strip())
        refs.append(reference.strip())

    # Compute ROUGE
    rouge_result = rouge.compute(predictions=preds, references=refs, use_stemmer=True)

    # Compute BERTScore
    bertscore_result = bertscore.compute(predictions=preds, references=refs, lang="en")

    # Aggregate results
    results = {
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
        "rougeLsum": rouge_result["rougeLsum"],
        "bertscore_f1": np.mean(bertscore_result["f1"]),
    }
    return results

# Example usage
results = evaluate_agent(agent, val_data, num_samples=200)
print("📊 Agent Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")
